In [ ]:
import argparse
import numpy as np

from pydrake.all import (DiagramBuilder, FloatingBaseType, RigidBodyPlant,
                         RigidBodyTree, Simulator, VectorSystem)
from underactuated import (FindResource, PlanarRigidBodyVisualizer,
                           SliderSystem)

tree = RigidBodyTree(FindResource("acrobot/acrobot.urdf"),
                     FloatingBaseType.kFixed)

%matplotlib

builder = DiagramBuilder()
acrobot = builder.AddSystem(RigidBodyPlant(tree))

visualizer = builder.AddSystem(PlanarRigidBodyVisualizer(tree,
                                                         xlim=[-4., 4.],
                                                         ylim=[-4., 4.]))
builder.Connect(acrobot.get_output_port(0), visualizer.get_input_port(0))

ax = visualizer.fig.add_axes([.2, .95, .6, .025])
torque_system = builder.AddSystem(SliderSystem(ax, 'Torque', -5., 5.))
builder.Connect(torque_system.get_output_port(0),
                acrobot.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.set_publish_every_time_step(False)

context = simulator.get_mutable_context()
context.SetContinuousState([1., 0., 0., 0.])

simulator.AdvanceTo(100000)

Using matplotlib backend: Qt5Agg


In [3]:
import math
import numpy as np

from pydrake.all import (DiagramBuilder, FloatingBaseType,
                         LinearQuadraticRegulator, RigidBodyTree,
                         Saturation, Simulator, WrapToSystem, SignalLogger)
from pydrake.examples.acrobot import (AcrobotInput, AcrobotPlant, AcrobotState)
from underactuated import (FindResource, PlanarRigidBodyVisualizer)

%matplotlib

def UprightState():
    state = AcrobotState()
    state.set_theta1(0.)
    state.set_theta2(0.)
    state.set_theta1dot(0.)
    state.set_theta2dot(0.)
    return state


def BalancingLQR():
    # Design an LQR controller for stabilizing the Acrobot around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original AcrobotState coordinates).

    acrobot = AcrobotPlant()
    context = acrobot.CreateDefaultContext()

    input = AcrobotInput()
    input.set_tau(0.)
    context.FixInputPort(0, input)

    context.get_mutable_continuous_state_vector()\
        .SetFromVector(UprightState().CopyToVector())

    Q = np.diag((10., 10., 1., 1.))
    R = [1]

    return LinearQuadraticRegulator(acrobot, context, Q, R)


if __name__ == "__main__":
    builder = DiagramBuilder()

    acrobot = builder.AddSystem(AcrobotPlant())
    saturation = builder.AddSystem(Saturation(min_value=[-10],
                                              max_value=[10]))
    builder.Connect(saturation.get_output_port(0), acrobot.get_input_port(0))
    wrapangles = WrapToSystem(4)
    wrapangles.set_interval(0, 0, 2.*math.pi)
    wrapangles.set_interval(1, -math.pi, math.pi)
    wrapto = builder.AddSystem(wrapangles)
    builder.Connect(acrobot.get_output_port(0), wrapto.get_input_port(0))
    controller = builder.AddSystem(BalancingLQR())
    builder.Connect(wrapto.get_output_port(0), controller.get_input_port(0))
    builder.Connect(controller.get_output_port(0),
                    saturation.get_input_port(0))
    

    state_logger = builder.AddSystem(SignalLogger(acrobot.get_output_port(0).size()))
    act_logger = builder.AddSystem(SignalLogger(controller.get_output_port(0).size()))
    builder.Connect(acrobot.get_output_port(0), state_logger.get_input_port(0))
    builder.Connect(saturation.get_output_port(0), act_logger.get_input_port(0))


    tree = RigidBodyTree(FindResource("acrobot/acrobot.urdf"),
                         FloatingBaseType.kFixed)
    visualizer = builder.AddSystem(PlanarRigidBodyVisualizer(tree,
                                                             xlim=[-4., 4.],
                                                             ylim=[-4., 4.]))
    builder.Connect(acrobot.get_output_port(0), visualizer.get_input_port(0))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.set_publish_every_time_step(False)
    context = simulator.get_mutable_context()

    for i in range(1):
        context.SetTime(0.)
        init_pert = 0.1*np.random.randn(4,)
        print(init_pert)
        context.SetContinuousState(UprightState().CopyToVector() + init_pert)
        simulator.Initialize()
        simulator.AdvanceTo(5)

Using matplotlib backend: Qt5Agg
[-0.09678927  0.0629509   0.1585581   0.1064886 ]
[ 0.18630748 -0.10297263 -0.06366959  0.04070918]


KeyboardInterrupt: 

[-0.0389305  -0.0486105   0.16965009 -0.02048805]
[-0.16137243  0.08915987  0.06315011 -0.3009472 ]
[-0.12272655  0.01749146 -0.02034932  0.16437832]


In [6]:
np.isnan(state_logger.data()).any()

False